# IMPUTING VALUES FOR THE NAN'S


# visitor_hist_starrating:  94.92 % & visitor_hist_adr_usd: 94.9 %
Imputing a single value didnt produce nice results. So I first imputed the mean of people from the same country going to the same destination. The remaining missing values were replaced by the mean of people from the same country going to the same country. After that I imputed the mean of the people from the same country. If there were any missing values left it was filled by the mean of everyone.

Note. I did a lot more testing but didnt add it to the notebook to make sure it is clear what i did. This procedure produced the best results.


# prop_review_score:  0.15 %
Imputing a single value worked for prop_review_score. The correlation between prop_review_score and click_bool is highest when we impute 0 for the missing values. 


# prop_location_score2:  21.99 %
It seems that the location_score2 is based on the search destination and the hotel location. It is probable that the further away the hotel is from the original search destination the lower the location_score2 is. Imputing a single value worked for prop_location_score2. The correlation between prop_location_score2 and click_bool is highest when we impute 0 for the missing values. 


# srch_query_affinity_score:  93.6 %
Imputing a single value didnt produce nice results. So I first imputed the mean of the property in other searches. The remaining missing values were replaced by the mean of people with the same site_id going to the same destination. After that I imputed the mean of the same site_id going to the same country. If there were any missing values left it was filled by the mean of everyone.


# orig_destination_distance:  32.43 %
Either a search has the distance for every result or for none. Imputing the max distance for all missing values gives the best result. This would increase the correlation of this attribute with the click_bool from r = .0023 to r = .0057. Although this correlation is still really weak, it is a simple, fast and effective solution to improve the quality of this attribute.


# Gross bookings USD: 97.21 %
An attribute that only occurs when a hotel is booked. Doesnt have much relevance to predicting clicks for now.


# Competitors: +- 80%
The ratings, invs, and perc difference columns were aggregrated. So from 24 columns reduced to 3. The ratings and invs columns have around 30% missing values, while the perc difference column has about 70% missing values. After some experimentation the following was imputed: for invs and perc difference the minimum was imputed and for ratings the mean was imputed. These values gave the most acceptable results.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('training_set_VU_DM_2014.csv')
df.shape

(4958347, 54)

In [3]:
print('\n\tPercentage of missing values:')
for column in df.columns:
    percentage = round((len(df[df[column].isnull()])/len(df))*100,2)
    if percentage > 0:
        print(column + ': ', percentage, '%')


	Percentage of missing values:
visitor_hist_starrating:  94.92 %
visitor_hist_adr_usd:  94.9 %
prop_review_score:  0.15 %
prop_location_score2:  21.99 %
srch_query_affinity_score:  93.6 %
orig_destination_distance:  32.43 %
comp1_rate:  97.58 %
comp1_inv:  97.39 %
comp1_rate_percent_diff:  98.1 %
comp2_rate:  59.17 %
comp2_inv:  57.04 %
comp2_rate_percent_diff:  88.78 %
comp3_rate:  69.06 %
comp3_inv:  66.7 %
comp3_rate_percent_diff:  90.46 %
comp4_rate:  93.8 %
comp4_inv:  93.07 %
comp4_rate_percent_diff:  97.36 %
comp5_rate:  55.18 %
comp5_inv:  52.4 %
comp5_rate_percent_diff:  83.04 %
comp6_rate:  95.16 %
comp6_inv:  94.74 %
comp6_rate_percent_diff:  98.06 %
comp7_rate:  93.64 %
comp7_inv:  92.81 %
comp7_rate_percent_diff:  97.21 %
comp8_rate:  61.34 %
comp8_inv:  59.92 %
comp8_rate_percent_diff:  87.6 %
gross_bookings_usd:  97.21 %


In [4]:
# First check the simplest solutions of imputing 1 value for all missing values in a column

from scipy.stats import pearsonr

for column in ['visitor_hist_starrating','visitor_hist_adr_usd', 'prop_review_score',
'prop_location_score2','srch_query_affinity_score','orig_destination_distance']:
    
    result = []
    df2 = df[[column, 'click_bool']]

    r1, p = pearsonr(df2[df2[column].notnull()][column], df2[df2[column].notnull()]['click_bool'])
    result.append((r1, 'old'))
    for option in ['mean', 'median', 'zero', 'firstq', 'thirdq', 'max', 'min']:
        df2 = df[[column, 'click_bool']]
        if option == 'mean':
            df2.loc[:,column] = df2[column].fillna(df2[column].mean())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'median':
            df2.loc[:,column] = df2[column].fillna(df2[column].median())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'zero':
            df2.loc[:,column] = df2[column].fillna(0)
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'firstq':
            df2.loc[:,column] = df2[column].fillna(df2[column].quantile(q = .25))
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'thirdq':
            df2.loc[:,column] = df2[column].fillna(df2[column].quantile(q = .75))
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'max':
            df2.loc[:,column] = df2[column].fillna(df2[column].max())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'min':
            df2.loc[:,column] = df2[column].fillna(df2[column].min())
            r, p = pearsonr(df2[column], df2['click_bool'])
        result.append((r, option))
        print(option, r)
    if r1 > 0:
        best_corr, best_option = max(result)
    elif r1 < 0:
        best_corr, best_option = min(result)
    if best_option != 'old':
        print(f"{column} had an original correlation of r = {round(r1, 5)} with the click_bool. \nBest option: impute {best_option} with r = {round(best_corr, 5)}")
        print()
    else:
        print(f"{column} had an original correlation of {round(r1, 5)} with the click_bool. \nThere was no better alternative.")
        print()

C:\Users\Mick\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


mean 0.0005694839393124071
median 0.000626779442498795
zero -0.0004418415627816627
firstq 0.00017204834390984516
thirdq 0.0008006384089318516
max 0.0007517801820802357
min -0.00034362416781758665
visitor_hist_starrating had an original correlation of 0.00254 with the click_bool. 
There was no better alternative.

mean 0.0008671475997370066
median 0.000736350692485108
zero 1.3140062061918199e-05
firstq 0.0004715742465620082
thirdq 0.0009906589213724986
max 0.0005793425650268978
min 1.3140062061918199e-05
visitor_hist_adr_usd had an original correlation of 0.00386 with the click_bool. 
There was no better alternative.

mean 0.023410036946974796
median 0.023394072813756547
zero 0.023444060320061386
firstq 0.023427803146072346
thirdq 0.0233524831940233
max 0.023303091176428058
min 0.023444060320061386
prop_review_score had an original correlation of r = 0.02342 with the click_bool. 
Best option: impute zero with r = 0.02344

mean 0.06901988487992158
median 0.07717329274108041
zero 0.083089

# Visitor History rating

In [ ]:
ptstar1 = pd.pivot_table(df, values='visitor_hist_starrating', index=['visitor_location_country_id', 'srch_destination_id'], aggfunc='mean').dropna().reset_index()
new_df = pd.merge(df.drop('visitor_hist_starrating', 1), ptstar1, how='left', on=['visitor_location_country_id', 'srch_destination_id'])
df.loc[:, 'visitor_hist_starrating'] = df['visitor_hist_starrating'].fillna(new_df['visitor_hist_starrating'])
del ptstar1, new_df

ptstar2 = pd.pivot_table(df, values='visitor_hist_starrating', index=['visitor_location_country_id', 'prop_country_id'], aggfunc='mean').dropna().reset_index()
new_df = pd.merge(df.drop('visitor_hist_starrating', 1), ptstar2, how='left', on=['visitor_location_country_id', 'prop_country_id'])
df.loc[:, 'visitor_hist_starrating'] = df['visitor_hist_starrating'].fillna(new_df['visitor_hist_starrating'])
del ptstar2, new_df

ptstar3 = pd.pivot_table(df, values='visitor_hist_starrating', index=['visitor_location_country_id'], aggfunc = 'mean').dropna().reset_index()
new_df = pd.merge(df.drop('visitor_hist_starrating', 1), ptstar3, how='left', on=['visitor_location_country_id'])
df.loc[:, 'visitor_hist_starrating'] = df['visitor_hist_starrating'].fillna(new_df['visitor_hist_starrating'])
del ptstar3, new_df

df.loc[:, 'visitor_hist_starrating'] = df['visitor_hist_starrating'].fillna(df['visitor_hist_starrating'].mean())

# Visitor history price usd

In [ ]:
ptprice1 = pd.pivot_table(df, values='visitor_hist_adr_usd', index=['visitor_location_country_id', 'srch_destination_id'], aggfunc='mean').dropna().reset_index()
new_df = pd.merge(df.drop('visitor_hist_adr_usd', 1), ptprice1, how='left', on=['visitor_location_country_id', 'srch_destination_id'])
df.loc[:, 'visitor_hist_adr_usd'] = df['visitor_hist_adr_usd'].fillna(new_df['visitor_hist_adr_usd'])
del ptprice1, new_df

ptprice2 = pd.pivot_table(df, values='visitor_hist_adr_usd', index=['visitor_location_country_id', 'prop_country_id'], aggfunc='mean').dropna().reset_index()
new_df = pd.merge(df.drop('visitor_hist_adr_usd', 1), ptprice2, how='left', on=['visitor_location_country_id', 'prop_country_id'])
df.loc[:, 'visitor_hist_adr_usd'] = df['visitor_hist_adr_usd'].fillna(new_df['visitor_hist_adr_usd'])
del ptprice2, new_df

ptprice3 = pd.pivot_table(df, values='visitor_hist_adr_usd', index=['visitor_location_country_id'], aggfunc = 'mean').dropna().reset_index()
new_df = pd.merge(df.drop('visitor_hist_adr_usd', 1), ptprice3, how='left', on=['visitor_location_country_id'])
df.loc[:, 'visitor_hist_adr_usd'] = df['visitor_hist_adr_usd'].fillna(new_df['visitor_hist_adr_usd'])
del ptprice3, new_df

df.loc[:, 'visitor_hist_adr_usd'] = df['visitor_hist_adr_usd'].fillna(df['visitor_hist_adr_usd'].mean())

# srch_query_affinity_score

In [ ]:
ptquery1 = pd.pivot_table(df, values='srch_query_affinity_score', index=['prop_id'], aggfunc='mean').dropna().reset_index()
new_df = pd.merge(df.drop('srch_query_affinity_score', 1), ptquery1, how='left', on=['prop_id'])
df.loc[:, 'srch_query_affinity_score'] = df['srch_query_affinity_score'].fillna(new_df['srch_query_affinity_score'])
del ptquery1, new_df

ptquery2 = pd.pivot_table(df, values='srch_query_affinity_score', index=['site_id', 'srch_destination_id'], aggfunc='mean').dropna().reset_index()
new_df = pd.merge(df.drop('srch_query_affinity_score', 1), ptquery2, how='left', on=['site_id', 'srch_destination_id'])
df.loc[:, 'srch_query_affinity_score'] = df['srch_query_affinity_score'].fillna(new_df['srch_query_affinity_score'])
del ptquery2, new_df

ptquery3 = pd.pivot_table(df, values='srch_query_affinity_score', index=['site_id', 'prop_country_id'], aggfunc = 'mean').dropna().reset_index()
new_df = pd.merge(df.drop('srch_query_affinity_score', 1), ptquery3, how='left', on=['site_id', 'prop_country_id'])
df.loc[:, 'srch_query_affinity_score'] = df['srch_query_affinity_score'].fillna(new_df['srch_query_affinity_score'])
del ptquery3, new_df

df.loc[:, 'srch_query_affinity_score'] = df['srch_query_affinity_score'].fillna(df['srch_query_affinity_score'].mean())

# Competitor information

In [5]:
# First check the simplest solutions of imputing 1 value for all missing values in a column

from scipy.stats import pearsonr

for column in ['comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff']:
    
    result = []
    df2 = df[[column, 'click_bool']]

    r1, p = pearsonr(df2[df2[column].notnull()][column], df2[df2[column].notnull()]['click_bool'])
    result.append((r1, 'old'))
    for option in ['mean', 'median', 'zero', 'firstq', 'thirdq', 'max', 'min']:
        df2 = df[[column, 'click_bool']]
        if option == 'mean':
            df2.loc[:,column] = df2[column].fillna(df2[column].mean())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'median':
            df2.loc[:,column] = df2[column].fillna(df2[column].median())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'zero':
            df2.loc[:,column] = df2[column].fillna(0)
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'firstq':
            df2.loc[:,column] = df2[column].fillna(df2[column].quantile(q = .25))
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'thirdq':
            df2.loc[:,column] = df2[column].fillna(df2[column].quantile(q = .75))
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'max':
            df2.loc[:,column] = df2[column].fillna(df2[column].max())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'min':
            df2.loc[:,column] = df2[column].fillna(df2[column].min())
            r, p = pearsonr(df2[column], df2['click_bool'])

        result.append((r, option))
    if r1 > 0:
        best_corr, best_option = max(result)
    elif r1 < 0:
        best_corr, best_option = min(result)
    if best_option != 'old':
        print(f"{column} had an original correlation of r = {round(r1, 5)} with the click_bool. \nBest option: impute {best_option} with r = {round(best_corr, 5)}")
        print()
    else:
        print(f"{column} had an original correlation of {round(r1, 5)} with the click_bool. \nThere was no better alternative.")
        print()

C:\Users\Mick\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


comp1_rate had an original correlation of 0.00794 with the click_bool. 
There was no better alternative.

comp1_inv had an original correlation of -0.00726 with the click_bool. 
There was no better alternative.

comp1_rate_percent_diff had an original correlation of 0.00213 with the click_bool. 
There was no better alternative.

comp2_rate had an original correlation of 0.01695 with the click_bool. 
There was no better alternative.

comp2_inv had an original correlation of r = -0.00028 with the click_bool. 
Best option: impute min with r = -0.00441

comp2_rate_percent_diff had an original correlation of 0.00139 with the click_bool. 
There was no better alternative.

comp3_rate had an original correlation of 0.01563 with the click_bool. 
There was no better alternative.

comp3_inv had an original correlation of -0.00672 with the click_bool. 
There was no better alternative.

comp3_rate_percent_diff had an original correlation of r = -0.0018 with the click_bool. 
Best option: impute max 

# Some comments
As expected imputing the one value for all missing values doesnt seem to work well. For many there are no better alternatives, and for some there are. But for the ones with a better alternative the correlation with click_bool is still extremely low. Next i will try to combine the info of the competitors into less variables.

In [6]:
# The mean for the 3 different attributes are taken based on the competitors that dont have missing values

NUMBER_INSTANCES = len(df)

rates = ["comp1_rate", "comp2_rate", "comp3_rate", "comp4_rate",
         "comp5_rate", "comp6_rate", "comp7_rate", "comp8_rate"]

invs = ['comp1_inv', 'comp2_inv', 'comp3_inv', 'comp4_inv',
       'comp5_inv', 'comp6_inv', 'comp7_inv', 'comp8_inv']


diffs = ['comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff',
        'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff',
        'comp7_rate_percent_diff', 'comp8_rate_percent_diff']


dfrates = df[rates]
dfinvs = df[invs]
dfdiffs = df[diffs]

ratemean = []
invsmean = []
diffsmean =[]
j = 0
for i in range(len(dfrates))[:NUMBER_INSTANCES]:
    rates = []
    for value in dfrates.values[i]:
        if value in [-1, 0, 1]:
            rates.append(value)
    
    if len(rates) == 0:
        ratemean.append(np.nan)
    else:
        ratemean.append(sum(rates)/len(rates))
        
    invs = []
    for value in dfinvs.values[i]:
        if value in [-1, 0, 1]:
            invs.append(value)
    
    if len(invs) == 0:
        invsmean.append(np.nan)
    else:
        invsmean.append(sum(invs)/len(invs))
        
    diffs = []
    for value in dfdiffs.values[i]:
        if (value < 500):
            diffs.append(value)
    
    if len(diffs) == 0:
        diffsmean.append(np.nan)
    else:
        diffsmean.append(sum(diffs)/len(diffs))
    
    if (i != 0) and (i%(NUMBER_INSTANCES/100) == 0):
        j += 1
        print(j, '%', end = ' ')

invsmean = np.asarray(invsmean)
ratemean = np.asarray(ratemean)
diffsmean = np.asarray(diffsmean)

dfcompetitors = df[['click_bool']][:NUMBER_INSTANCES]
dfcompetitors.loc[:, 'invmean'] = invsmean
dfcompetitors.loc[:, 'ratemean'] = ratemean
dfcompetitors.loc[:, 'diffsmean'] = diffsmean
dfcompetitors.corr()

,click_bool,invmean,ratemean,diffsmean
click_bool,1.000000,-0.003433,0.023587,0.014690
invmean,-0.003433,1.000000,0.000526,0.033479
ratemean,0.023587,0.000526,1.000000,0.111519
diffsmean,0.014690,0.033479,0.111519,1.000000


In [7]:
print('\n\tPercentage of missing values:')
for column in dfcompetitors.columns:
    percentage = round((len(dfcompetitors[dfcompetitors[column].isnull()])/len(dfcompetitors))*100,2)
    if percentage > 0:
        print(column + ': ', percentage, '%')


	Percentage of missing values:
invmean:  32.39 %
ratemean:  34.62 %
diffsmean:  68.17 %


# More comments
So using the original 24 features resulted in very low correlations with the click_bool. The aggregrated features have a higher correlations with the click_bool, and they have less missing values (from around 80% to to 30% for inv and rates and 66% for percentage difference). Next ill try to see if imputing 1 value for all missing values is possible.

In [8]:
# First check the simplest solutions of imputing 1 value for all missing values in a column

from scipy.stats import pearsonr

for column in ['invmean', 'ratemean', 'diffsmean']:
    
    result = []
    df2 = dfcompetitors[[column, 'click_bool']]

    r1, p = pearsonr(df2[df2[column].notnull()][column], df2[df2[column].notnull()]['click_bool'])
    result.append((r1, 'old'))
    
    for option in ['mean', 'median', 'zero', 'firstq', 'thirdq', 'max', 'min']:
        df2 = dfcompetitors[[column, 'click_bool']]
        if option == 'mean':
            df2.loc[:,column] = df2[column].fillna(df2[column].mean())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'median':
            df2.loc[:,column] = df2[column].fillna(df2[column].median())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'zero':
            df2.loc[:,column] = df2[column].fillna(0)
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'firstq':
            df2.loc[:,column] = df2[column].fillna(df2[column].quantile(q = .25))
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'thirdq':
            df2.loc[:,column] = df2[column].fillna(df2[column].quantile(q = .75))
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'max':
            df2.loc[:,column] = df2[column].fillna(df2[column].max())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'min':
            df2.loc[:,column] = df2[column].fillna(df2[column].min())
            r, p = pearsonr(df2[column], df2['click_bool'])
        print(option, r)
        result.append((r, option))
    if r1 > 0:
        best_corr, best_option = max(result)
    elif r1 < 0:
        best_corr, best_option = min(result)
    if best_option != 'old':
        print(f"{column} had an original correlation of r = {round(r1, 5)} with the click_bool. \nBest option: impute {best_option} with r = {round(best_corr, 5)}")
        print()
    else:
        print(f"{column} had an original correlation of {round(r1, 5)} with the click_bool. \nThere was no better alternative.")
        print()

C:\Users\Mick\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


mean -0.0028119046683387308
median -0.0030685901840169483
zero -0.0030685901840169483
firstq -0.0030685901840169483
thirdq -0.0030685901840169483
max 0.0012642007549602935
min -0.003343811441173417
invmean had an original correlation of -0.00343 with the click_bool. 
There was no better alternative.

mean 0.018965622495426035
median 0.018866750640488584
zero 0.018866750640488584
firstq 0.018866750640488584
thirdq 0.018866750640488584
max 0.01413901599955859
min 0.008288520911911764
ratemean had an original correlation of 0.02359 with the click_bool. 
There was no better alternative.

mean 0.00840734215356335
median 0.009178891821516679
zero 0.00946734385087474
firstq 0.009506568459263183
thirdq 0.007899222781885476
max -0.004113221370566694
min 0.009523835130745723
diffsmean had an original correlation of 0.01469 with the click_bool. 
There was no better alternative.



# Conclusion and further steps
For the invmean it would be usefull to impute the minimum for all missing values since the reduction in the correlation with click_bool is 0.0001 which is almost nothing. 

For the ratemean we can decide to impute the mean, which would give a reduction of .004 with click_bool. This is still not a big difference but we have to be carefull when deciding this.

The min of diffsmean can be imputed since it gives a reduction of .005 with click_bool.

In [11]:
# What happens when we concatenate the two dataframes and fill in the ratemean and diffsmean and compute rate*diff
dfcompetitors2 = df[['click_bool']]
dfcompetitors2.loc[:, 'ratemean'] = dfcompetitors['ratemean'].fillna(dfcompetitors['ratemean'].mean())
dfcompetitors2.loc[:, 'invmean'] = dfcompetitors['invmean'].fillna(dfcompetitors['ratemean'].min())
dfcompetitors2.loc[:, 'diffsmean'] = dfcompetitors['diffsmean'].fillna(dfcompetitors['diffsmean'].min())
dfcompetitors2.corr()

C:\Users\Mick\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Mick\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,click_bool,ratemean,invmean,diffsmean
click_bool,1.000000,0.018966,-0.003344,0.009524
ratemean,0.018966,1.000000,0.000112,0.115153
invmean,-0.003344,0.000112,1.000000,0.241298
diffsmean,0.009524,0.115153,0.241298,1.000000


In [12]:
dfcompetitors2.drop('click_bool', 1).to_csv('Dataset_Competitors.csv')

# Lets create a new df with no NAN-values
Except for the visitor history rating/price :)

In [ ]:
finaldf = df[['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'click_bool', 
       'gross_bookings_usd', 'booking_bool']]

finaldf.loc[:, 'prop_review_score'] = finaldf['prop_review_score'].fillna(0)
print('prop_review_score = done')
finaldf.loc[:, 'prop_location_score2'] = finaldf['prop_location_score2'].fillna(0)
print('prop_location_score2 = done') 
finaldf.loc[:, 'orig_destination_distance'] = finaldf['orig_destination_distance'].fillna(finaldf['orig_destination_distance'].max())
print('orig_desination_distance = done')
dfcompetitors2 = dfcompetitors2.drop('click_bool', 1)
finaldf = pd.concat([finaldf, dfcompetitors2], axis=1, join_axes=[finaldf.index])
print('competitors info = done')

In [ ]:
finaldf.describe()

In [ ]:
finaldf.to_csv('Dataset_complete.csv')